# Using PySpark to Create an Amazon Review Recommendation System 



This Jupyter Notebook contains code to create a recommendation system for Amazon user reviews on specific products using PySpark.  It was created as a final project for the class INFO 607: Applied Database Technologies at Drexel University.  The data was downloaded from FIXME: data source.  

Additional documentation on this project can be found at the Github repository [here](https://github.com/zachcarlson/ProductRecommender).

## Configuration

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
INPUT_DIRECTORY = "/content/drive/MyDrive/Grad School/INFO 607/ProductRecommender/data/" #for google mount
# INPUT_DIRECTORY = "./data/" #for jupyter notebook

The cell below may take 1-2 minutes to execute:

In [3]:
%%capture 
#prevent large printout with %%capture

#Download Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

#Install Apache Spark 3.2.1 with Hadoop 3.2, get zipped folder
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz

#Unzip folder
!tar xvf spark-3.2.1-bin-hadoop3.2.tgz

#Install findspark, pyspark 3.2.1
!pip install -q findspark
!pip install pyspark==3.2.1

#Set variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.2.1-bin-hadoop3.2"

## Load Packages

In [4]:
import pandas as pd
import pyspark.sql.functions as F

## Data Acquisition, Preprocessing

### Import Data

In [5]:
#create SparkSession and SparkContext objects
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()

In [6]:
#Import data
file_path = INPUT_DIRECTORY + "ratings_electronics.csv"
ratings = spark.read.csv(file_path, header=False, inferSchema=True)
ratings.show(5)

+--------------+----------+---+----------+
|           _c0|       _c1|_c2|       _c3|
+--------------+----------+---+----------+
| AKM1MP6P0OYPR|0132793040|5.0|1365811200|
|A2CX7LUOHB2NDG|0321732944|5.0|1341100800|
|A2NWSAGRHCP8N5|0439886341|1.0|1367193600|
|A2WNBOD3WNDNKT|0439886341|3.0|1374451200|
|A1GI0U4ZRJA8WN|0439886341|1.0|1334707200|
+--------------+----------+---+----------+
only showing top 5 rows



In [7]:
ratings = ratings.withColumnRenamed("_c0", "reviewerID") \
                  .withColumnRenamed("_c1", "productID") \
                  .withColumnRenamed("_c2", "rating") \
                  .withColumnRenamed("_c3", "timestamp")
ratings.show(5)

+--------------+----------+------+----------+
|    reviewerID| productID|rating| timestamp|
+--------------+----------+------+----------+
| AKM1MP6P0OYPR|0132793040|   5.0|1365811200|
|A2CX7LUOHB2NDG|0321732944|   5.0|1341100800|
|A2NWSAGRHCP8N5|0439886341|   1.0|1367193600|
|A2WNBOD3WNDNKT|0439886341|   3.0|1374451200|
|A1GI0U4ZRJA8WN|0439886341|   1.0|1334707200|
+--------------+----------+------+----------+
only showing top 5 rows



## EDA

Count ratings made by each customer

In [9]:
ratings.select("reviewerID", "productID", "rating")\
        .groupby("reviewerID")\
        .count()\
        .sort("count", ascending = False)\
        .show()

+--------------+-----+
|    reviewerID|count|
+--------------+-----+
| A5JLAU2ARJ0BO|  520|
|  ADLVFFE4VBT8|  501|
|A3OXHLG6DIBRW8|  498|
|  A6FIAB28IS79|  431|
| A680RUE1FDO8B|  406|
|A1ODOGXEYECQQ8|  380|
|A36K2N527TXXJN|  314|
|A2AY4YUOX2N1BQ|  311|
| AWPODHOB4GFWL|  308|
| ARBKYIVNYWK3C|  296|
|A25C2M3QF9G7OQ|  296|
|A22CW0ZHY3NJH8|  292|
|A3EXWV8FNSSFL6|  282|
| A38RMU1Y5TDP9|  282|
|A3LGT6UZL99IW1|  279|
|A2NOW4U7W3F7RI|  277|
|A23GFTVIETX7DS|  270|
|A3PD8JD9L4WEII|  266|
|A17BUUBOU0598B|  261|
|A3AYSYSLHU26U9|  257|
+--------------+-----+
only showing top 20 rows



Count ratings for each product

In [12]:
ratings.select("reviewerID", "productID", "rating")\
        .groupby("productID")\
        .count()\
        .sort("count", ascending = False)\
        .show()

+----------+-----+
| productID|count|
+----------+-----+
|B0074BW614|18244|
|B00DR0PDNE|16454|
|B007WTAJTO|14172|
|B0019EHU8G|12285|
|B006GWO5WK|12226|
|B003ELYQGG|11617|
|B003ES5ZUU|10276|
|B007R5YDYA| 9907|
|B00622AG6S| 9823|
|B0002L5R78| 9487|
|B008OHNZI0| 8966|
|B003LR7ME6| 8840|
|B000LRMS66| 8715|
|B009SYZ8OC| 8370|
|B00BGA9WK2| 7561|
|B004QK7HI8| 7060|
|B009A5204K| 7059|
|B00BGGDVOO| 6893|
|B0098F5W0Q| 6616|
|B002MAPRYU| 6599|
+----------+-----+
only showing top 20 rows



Count ratings for each pair of product - customer.

NOTE: There is only one count for each pair.

In [13]:
ratings.select("reviewerID", "productID", "rating")\
        .groupby("productID","reviewerID")\
        .count()\
        .sort("count", ascending = True)\
        .show()

+----------+--------------+-----+
| productID|    reviewerID|count|
+----------+--------------+-----+
|0321732944|A2CX7LUOHB2NDG|    1|
|0439886341|A2NWSAGRHCP8N5|    1|
|0511189877|A3J3BRHTDRFJ2G|    1|
|0528881469|A19TBA1WARJS55|    1|
|0528881469|A1E4WG8HRWWK4R|    1|
|0528881469|A1H8PY3QHMQQA0|    1|
|0528881469|A24QFSUU00IZ05|    1|
|0528881469|A29LPQQDG7LD5J|    1|
|0528881469|A2O8FIJR9EBU56|    1|
|0528881469|A2XSWV6AQI90BR|    1|
|0528881469|A3C5SMBSKKWNPT|    1|
|0528881469|A3N7T0DY83Y4IG|    1|
|0528881469| AYTBGUX49LF3W|    1|
|059400232X|A22FB2WSZSXSHH|    1|
|0594012015|A1HOSS7PNC1LMU|    1|
|0594033926|A253JJFXQNPCOJ|    1|
|0594033926|A2PL24SVISXZM9|    1|
|0594033926|A3HQ8RXUMJ2Y58|    1|
|0594033926|A3I1C8WM8DLSMM|    1|
|0594033926| AHYURLVH267MA|    1|
+----------+--------------+-----+
only showing top 20 rows



Count and average ratings for each product

In [14]:
avg_ratings = (ratings
                .select("productID", "rating")              # Select Columns
                .groupby("productID")                       # Group by productID
                .agg(                           
                     F.count("rating").alias("Count"),      # Count number of ratings
                     F.avg("rating").alias("Average")       # Average ratings for each product
                     )
                .sort("Average", "Count", ascending = [False, False]) # Sort results by average and count
            )

In [15]:
avg_ratings.show()

+----------+-----+-------+
| productID|Count|Average|
+----------+-----+-------+
|B008I64AQ8|   45|    5.0|
|B00GQDCMVA|   41|    5.0|
|B007WMOCW6|   38|    5.0|
|B00592002W|   36|    5.0|
|B002T3JN0Y|   36|    5.0|
|B00DWLPUZY|   34|    5.0|
|B00KC7I2GU|   31|    5.0|
|B000FIJA6W|   28|    5.0|
|B00009R8ZW|   28|    5.0|
|B0002VAVII|   25|    5.0|
|B0007KK2KG|   23|    5.0|
|B00KSLCU72|   22|    5.0|
|B001R84CGW|   22|    5.0|
|B004PLVROS|   22|    5.0|
|B00AN8Z7B6|   22|    5.0|
|B00GT3EDRS|   21|    5.0|
|B00G9W9LJY|   20|    5.0|
|B00CRN3D98|   20|    5.0|
|B007ECEHBK|   20|    5.0|
|B00AO0K42U|   20|    5.0|
+----------+-----+-------+
only showing top 20 rows



Replacing productID & reviewerID with numbers starting at 1:

In [16]:
# Add an integer id for reviewer
reviewers = ratings.select("reviewerID").distinct().createOrReplaceTempView('reviewers')
new_rev_id = spark.sql('select row_number() over (order by "reviewerID") as reviewer_id, * from reviewers')

# Add an integer id for product
products = ratings.select("productID").distinct().createOrReplaceTempView('products')
new_prod_id = spark.sql('select row_number() over (order by "productID") as product_id, * from products')

# Join dfs
new_ratings = ratings.join(new_rev_id, "reviewerID", "inner").join(new_prod_id, "productID", "inner")

# Sanity Check
new_ratings.show()

+----------+--------------+------+----------+-----------+----------+
| productID|    reviewerID|rating| timestamp|reviewer_id|product_id|
+----------+--------------+------+----------+-----------+----------+
|0528881469|A1H8PY3QHMQQA0|   2.0|1290556800|     250975|       930|
|0528881469|A265MKAR2WEH3Y|   4.0|1294790400|     397298|       930|
|0528881469|A29LPQQDG7LD5J|   1.0|1352073600|      73203|       930|
|0528881469|A3IQGFB959IR4P|   1.0|1327363200|     135306|       930|
|0528881469|A3N7T0DY83Y4IG|   3.0|1283990400|     124959|       930|
|0528881469| AR84FMFYCQCWF|   1.0|1296691200|     145834|       930|
|0528881469|A1DA3W4GTFXP6O|   5.0|1405641600|     762649|       930|
|0528881469| A2CPBQ5W4OGBX|   2.0|1277078400|     710366|       930|
|0528881469|A2O8FIJR9EBU56|   4.0|1278547200|     501568|       930|
|0528881469|A3C5SMBSKKWNPT|   5.0|1285113600|     741686|       930|
|0528881469| AMO214LNFCEI4|   1.0|1290643200|     783563|       930|
|0528881469| AO94DHGC771SJ|   5.0|

In [17]:
new_prod_id.show(), new_rev_id.show()

+----------+----------+
|product_id| productID|
+----------+----------+
|         1|7793224531|
|         2|9966694242|
|         3|9967222247|
|         4|9985975413|
|         5|9990950369|
|         6|B000000O3J|
|         7|B00000J025|
|         8|B00000J0IV|
|         9|B00000J1PX|
|        10|B00000J3Q1|
|        11|B00000J3SV|
|        12|B00000J47G|
|        13|B00000JBK6|
|        14|B00001W0DC|
|        15|B00002S73F|
|        16|B00003CWGY|
|        17|B00003OPEV|
|        18|B00004SC3R|
|        19|B00004TL5X|
|        20|B00004U0EB|
+----------+----------+
only showing top 20 rows

+-----------+--------------+
|reviewer_id|    reviewerID|
+-----------+--------------+
|          1| AFS5ZGZ2M3ZGV|
|          2|A18FTRFQQ141CP|
|          3|A2GPNXFUUV51ZZ|
|          4|A3GF7FD6065R2H|
|          5|A16W4IDX9O70NU|
|          6| A4035XND6J8CS|
|          7|A15K7HV1XD6YWR|
|          8|A3FZ6D8NP9775P|
|          9| AA36DB7PNNJP2|
|         10|A3PDGWYC08DXF4|
|         11|A37JOONB

(None, None)

## Recommendation System

## Conclusions